In [ ]:
#импортируем библиотеки
import requests #чтобы получать код страница сайта
from bs4 import BeautifulSoup #чтобы "ходить" по тэгам,доставать нужную нам информацию
import pandas as pd #для работы с таблицами и файлами
from time import sleep #для выставления задержек

In [ ]:
#тут мы парсим кинопоиск, а именно результаты по запросу "секс". 
#наша задача собрать все названия фильмов со словом "секс" на русском языке, на языке оригинала, год выхода фильма и страну


data = [] #создаем список для хранения наших данных

for i in range(1,7): #тут прописываем сколько ссылок нужно пройти. в нашем случае их шесть (плюс одна) и начинаем мы с 1
    url = 'https://www.kinopoisk.ru/s/type/film/list/1/find/%F1%E5%EA%F1/order/relevant/page/'+ str(i)+'/' #генерируем нашу ссылку на страницу
    r = requests.get(url) #делаем запрос к сайту
    sleep(0.5) #ставим задержку в половину секунды, чтобы не слишком мучить сайт

    soup = BeautifulSoup(r.text, 'lxml') #если все хорошо, то мы получаем в ответ код страницы и передаем его нашему супу

    films = soup.findAll('div', class_='element') #находим все теги, в которых лежит инфа по фильмам, получаем их в виде списка

    for f in films: #начинаем перебирать эти теги и уточнять, что именно из них нам нужно достать
        russian_name = f.find('div', class_='info').find('p').find('a').text #достаем имя на русском
        #print(russian_name)

        origin_name = f.find('div', class_='info').find('span', class_='gray').text #достаем оригинальное имя
        #print(origin_name)

        year = f.find('div', class_='info').find('span').text #достаем год
        #print(year)

        country = f.find('div', class_='info').findAll('span', class_='gray')[1].text.split(',')[0] #достаем страну произдводства
        #print(country)
        #print('-----')

        data.append([russian_name, origin_name, year, country]) #записываем все наши данные в список

              
    
header = ['russian_name', 'origin_name', 'year', 'country'] #тут у нас лежат названия наших колонок
#а ниже просто сохраняем наш список в файл
df = pd.DataFrame(data, columns=header)   
df.to_csv('ТУТ ВАША ДИРЕКТОРИЯ/kinopoisk_data.csv', sep=';', encoding='utf8')

In [ ]:
#это просто еще один метод сохранения файлов с помощью библиотеки csv (для этого нужно импортировать ее вверху import csv)
import csv 

csv_file = open('ТУТ ВАША ДИРЕКТОРИЯ\ПУТЬ', 'w')
writer = csv.writer(csv_file, delimiter=';')
writer.writerow(head)

for row in data:
    writer.writerow(row)

csv_file.close() 

In [ ]:
#а тут мы парсим страничку кинопоиска ТОР-250 фильмов, собираем название фильма, год и ссылку на страничку фильма

data = []

for i in range(1,10):
    url = 'https://www.kinopoisk.ru/lists/top250/?page=' + str(i) + '&tab=all'
    r = requests.get(url)
    sleep(5) #тут можно увеличивать задержку, смотря как кинопоиск будет вас банить
    
    
    soup = BeautifulSoup(r.text, 'lxml')

    films = soup.findAll('div', class_='desktop-rating-selection-film-item')

    for f in films:
        name = f.find('div', class_='selection-film-item-meta selection-film-item-meta_theme_desktop').find('p').text
        year = f.find('div', class_='selection-film-item-meta selection-film-item-meta_theme_desktop').find('p', class_='selection-film-item-meta__original-name').text

        link = 'https://www.kinopoisk.ru'+f.find('a', class_='selection-film-item-meta__link').get('href') # вот тут мы получаем ссылку на страничку фильм. ищем тэг а, а потом с помощью команды get достаем href


        data.append([name, year, link])


    

In [ ]:
#а здесь мы ходим по нашему списку, берем из него ссылку на каждую страницу фильма, открываем эту страницу и достаем оценку кинопоиска и imdb
new_data = [] #создадим новый пустой список, куда добавим уже спарсенный данные name, year, link + добавим новые kinopoisk_rate, imdb_rate
for i in data:
    link = i[2]
    r = requests.get(link)
    sleep(25)
    
    soup = BeautifulSoup(r.text, 'lxml')
    
    kinopoisk_rate = soup.find('a', class_='film-rating-value styles_rootPositive__ac3xv styles_rootLink__1CSPc').text
    imdb_rate = soup.find('span', class_='styles_valueSection__19woS').text.replace('IMDb: ', '')
    
    print(i[0])
    print(kinopoisk_rate)
    print(imdb_rate)
    print('-----')
    
    row = [i[0], i[1], i[2], kinopoisk_rate, imdb_rate]
    new_data.append(row)
    